In [1]:
from datetime import datetime, timedelta

In [2]:
import requests

In [3]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import plotly.express as px

In [4]:
from IPython.display import clear_output

In [5]:
import re

In [6]:
import pickle
with open('code_dic.pkl',mode='rb') as f:
    code_dic = pickle.load(f)

### データ更新用（毎週末）

In [7]:
pos_data = pd.read_excel('cftc先物ポジション_current.xlsx').iloc[:,1:]
pos_data.tail()
#pos_data.columns = ['date','long','short']
pos_data.tail()
#pos_data[pos_data['date']==20221004]

,date,trader,commodity,long,short
85370,20240806,LeveragedFunds,E-MINI S&P FINANCIAL INDEX - CHICAGO MERCANTIL...,920,1997
85371,20240806,AssetManager,E-MINI S&P FINANCIAL INDEX - CHICAGO MERCANTIL...,20382,430
85372,20240806,Dealer,E-MINI S&P FINANCIAL INDEX - CHICAGO MERCANTIL...,30507,51417
85373,20240806,AssetManager,E-MINI S&P TECHNOLOGY INDEX - CHICAGO MERCANTI...,8507,0
85374,20240806,NonRep,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,28694,40996


In [8]:
pos_data.shape[0]

85375

In [9]:
pos_data.dtypes

date          int64
trader       object
commodity    object
long          int64
short         int64
dtype: object

In [10]:
today = datetime.today()

In [11]:
tuesdays = []
date = str(pos_data['date'].tolist()[-1])
date = datetime.strptime(date,'%Y%m%d')+ timedelta(days=7)
while True:
    if date > today:
        break
    else:
        # 残りの火曜日を見つける
        tuesdays.append(date.strftime('%Y%m%d'))
    date += timedelta(days=7)

In [12]:
tuesdays

['20240813', '20240820']

In [13]:
#tuesdays = ['20221004']

In [14]:
datetime.weekday(datetime.today())

5

In [15]:
missed = []
i = 1
for date in tuesdays:
    if datetime.today() < datetime.strptime(date,'%Y%m%d')+timedelta(days=4):
        print('future')
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/financial_lf"+date[-4:]+yyyy[-2:]+".htm"
    print(url)
    #url = 'https://www.cftc.gov/dea/futures/financial_lf.htm'
    
    texts = requests.get(url).text
    pattern = r'<!--ih:includeHTML file="financiallf.txt"-->([^"]+)<!--/ih:includeHTML-->'
    texts = re.findall(pattern, texts)[0]
    texts = texts.split('\r\n')
    texts = [s for s in texts if (s != '')&(s != ' ')]
    #if date == '20220517':
    #    texts[0] ='USD Malaysian Crude Palm Oil C - CHICAGO MERCANTILE EXCHANGE         Code-037021'

    pos = [item for i, element in enumerate(texts) if 'Traders in Financial Futures' in element for item in ((i-1, i) if i > 0 else (i,))]
    #print(pos)
    nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
    #[print(s[9]) for s in nested_list]
    for s in nested_list:
        datas = [int(ss) for ss in s[9].replace(',','').split(' ') if not ss == '']
        #print(datas)
        traders = ['Dealer','AssetManager','LeveragedFunds','Other','NonRep']

        if (not s[6].startswith(' '))&(s[7].split('  ')[0] not in list(code_dic.keys())):
            #print(s[7].split('  ')[0])
            #print(s[6])
            code_dic[s[7].split('  ')[0]]=s[6]
        elif (s[6].startswith(' '))&(s[7].split('  ')[0] in list(code_dic.keys())):
            s[6] = code_dic[s[7]]
            #print(s[6])
            #break
            
        k = 0
        for j in range(0,len(datas),3):
            data = datas[j:j+2]
            #print(date,traders[k],s[6],data)
            temp = pd.DataFrame([date,traders[k],s[6]]+data).T
            temp.columns=['date','trader','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
            k += 1

    print(str(i)+'/'+str(len(tuesdays)))
    #clear_output(True)
    i += 1
    
print('終わりました')

https://www.cftc.gov/sites/default/files/files/dea/cotarchives/2024/futures/financial_lf081324.htm
1/2
https://www.cftc.gov/sites/default/files/files/dea/cotarchives/2024/futures/financial_lf082024.htm
2/2
終わりました


In [16]:
missed

[]

In [17]:
#pos_data[pos_data['date']==20221004]
pos_data = pos_data.reset_index(drop=True)
pos_data.tail()

,date,trader,commodity,long,short
85960,20240820,Dealer,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,56486,27035
85961,20240820,AssetManager,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,34139,55690
85962,20240820,LeveragedFunds,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,63984,88786
85963,20240820,Other,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,15395,1554
85964,20240820,NonRep,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,32701,29640


In [18]:
pos_data.to_excel('cftc先物ポジション_current.xlsx')

### データベースの構築用

In [15]:

def find_first_tuesday(year):
    # その年の1月1日
    date = datetime(year, 1, 1)
    # 1月1日が火曜日になるまで日付を進める
    while date.weekday() != 1:
        date += timedelta(days=1)
    return date

def find_tuesdays(start_year, end_year):
    tuesdays = []
    # 最初の火曜日を見つける
    date = find_first_tuesday(start_year)

    while True:
        if date.year == end_year + 1:
            break
        # 残りの火曜日を見つける
        tuesdays.append(date.strftime('%Y%m%d'))
        date += timedelta(days=7)  # 次の火曜日

    return tuesdays

In [16]:
def find_yen(arg_list):
    key1 = 'JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE                           Code-097741'
    for i in range(len(arg_list)):
        text = arg_list[i]
        if text == key1:
            data = [s for s in arg_list[i+9].split(' ') if not s == '']
            return [int(data[0].replace(',','')),int(data[1].replace(',','')),int(data[0].replace(',','')) - int(data[1].replace(',',''))]
        else:
            pass

In [17]:

# 関数を実行するための年を指定
start_year = 2018
end_year = 2024

# 結果を取得
tuesdays = find_tuesdays(start_year, end_year)

today = datetime.today().year

In [18]:
#tuesdays = ['20240220','20240227']

https://www.cftc.gov/sites/default/files/files/dea/cotarchives/2022/futures/financial_lf052422.htm

In [19]:
#tuesdays = ['20240220']
missed = ['20181225','20190101','20191224','20201222','20230704'] # 本来の火曜日
adj =  ['20181224','20181231','20191230','20201221','20230703'] # CFTCに登録された日付
tuesdays = [adj[missed.index(tuesdays[i])] if tuesdays[i] in missed else tuesdays[i] for i in range(len(tuesdays))]
tuesdays = sorted(tuesdays) # 登録ベースの火曜日

In [25]:
tuesdays[-4:]

['20241210', '20241217', '20241224', '20241231']

In [37]:
datetime.today()+timedelta(days = 5-datetime.today().weekday())

datetime.datetime(2024, 8, 17, 8, 56, 0, 764853)

In [39]:
temp = [s for s in tuesdays if datetime.today() > datetime.strptime(s,'%Y%m%d')+timedelta(days = 6 - datetime.today().weekday())]
temp[-4:]

['20240716', '20240723', '20240730', '20240806']

In [20]:
[print(s) for s in missed if s in tuesdays]

[]

In [40]:
pos_data = pd.DataFrame(columns=['date','trader','commodity','long','short'])
code_dic = {"a":""}
i = 1
for date in tuesdays:
    if datetime.today() <= datetime.strptime(date,'%Y%m%d')+timedelta(days = 6 - datetime.today().weekday()):
        print('公表済みの処理は終わりました。')
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/financial_lf"+date[-4:]+yyyy[-2:]+".htm"
    
    texts = requests.get(url).text
    pattern = r'<!--ih:includeHTML file="financiallf.txt"-->([^"]+)<!--/ih:includeHTML-->'
    texts = re.findall(pattern, texts)[0]
    texts = texts.split('\r\n')
    texts = [s for s in texts if (s != '')&(s != ' ')]

    pos = [item for i, element in enumerate(texts) if 'Traders in Financial Futures' in element for item in ((i-1, i) if i > 0 else (i,))]
    #print(pos)
    nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
    #[print(s[9]) for s in nested_list]
    for s in nested_list:
        datas = [int(ss) for ss in s[9].replace(',','').split(' ') if not ss == '']
        #print(datas)
        traders = ['Dealer','AssetManager','LeveragedFunds','Other','NonRep']

        if (not s[6].startswith(' '))&(s[7].split('  ')[0] not in list(code_dic.keys())):
            code_dic[s[7].split('  ')[0]]=s[6]
        elif (s[6].startswith(' '))&(s[7].split('  ')[0] in list(code_dic.keys())):
            s[6] = code_dic[s[7].split('  ')[0]]
        k = 0
        for j in range(0,len(datas),3):
            data = datas[j:j+2]
            #print(date,traders[k],s[6],data)

            # 登録ベースを本来の日付に変更
            if date in adj:
                date = missed[adj.index(date)]
                
            temp = pd.DataFrame([date,traders[k],s[6]]+data).T
            temp.columns=['date','trader','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
            k += 1

    print(str(i)+'/'+str(len(tuesdays)))
    clear_output(True)
    i += 1
print('終わりました')

公表済みの処理は終わりました。
終わりました


In [41]:
pos_data.tail()

,date,trader,commodity,long,short
0,20240806,Dealer,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,116832,49207
0,20240806,AssetManager,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,48231,46293
0,20240806,LeveragedFunds,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,66894,137437
0,20240806,Other,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,20027,6745
0,20240806,NonRep,VIX FUTURES - CBOE FUTURES EXCHANGE ($1000 X...,28694,40996


In [42]:
pos_data.head()

,date,trader,commodity,long,short
0,20180102,Dealer,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE ...,3592,56761
0,20180102,AssetManager,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE ...,42615,9406
0,20180102,LeveragedFunds,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE ...,32395,32533
0,20180102,Other,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE ...,5713,1758
0,20180102,NonRep,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE ...,35774,19631


In [43]:
pos_data.dtypes

date         object
trader       object
commodity    object
long         object
short        object
dtype: object

In [44]:
pos_data['date']=pos_data['date'].astype('str')
pos_data['trader']=pos_data['trader'].astype('str')
pos_data['commodity']=pos_data['commodity'].astype('str')
pos_data['long']=pos_data['long'].astype('int')
pos_data['short']=pos_data['short'].astype('int')
pos_data.dtypes

date         object
trader       object
commodity    object
long          int64
short         int64
dtype: object

In [45]:
# チェック
temp = list(set(pos_data['date'].tolist()))
chk = [temp[i] for i in range(len(temp)) if temp[i] not in tuesdays]
missed == sorted(list(set(chk)))

True

In [46]:
pos_data = pos_data.sort_values('date')
pos_data.to_excel('cftc先物ポジション_current.xlsx')

In [47]:
import pickle

with open('code_dic.pkl',mode='wb') as f:
    pickle.dump(code_dic,f)

#### ここまで

In [ ]:
missed2 = []
i = 1
for date in adj:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/financial_lf"+date[-4:]+yyyy[-2:]+".htm"
    try:
        texts = requests.get(url).text
        pattern = r'<!--ih:includeHTML file="financiallf.txt"-->([^"]+)<!--/ih:includeHTML-->'
        texts = re.findall(pattern, texts)[0]
        texts = texts.split('\r\n')
        texts = [s for s in texts if (s != '')&(s != ' ')]
        pos = [item for i, element in enumerate(texts) if 'Traders in Financial Futures' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
        nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
        for s in nested_list:
            datas = [int(ss) for ss in s[9].replace(',','').split(' ') if not ss == '']
            #print(datas)
            traders = ['Dealer','AssetManager','LeveragedFunds','Other','NonRep']
            k = 0
            for j in range(0,len(datas),3):
                data = datas[j:j+2]
                #print(date,traders[k],s[6],data)
                temp = pd.DataFrame([date,traders[k],s[6]]+data).T
                temp.columns=['date','trader','commodity','long','short']
                pos_data = pd.concat([pos_data,temp],axis=0)
                k += 1
    except:
        print(date)
        missed2 += [date]
        pass
    print(str(i)+'/'+str(len(adj)))
    clear_output(True)
    i += 1
print('終わりました')

In [ ]:
pos_data = pos_data.sort_values('date')

In [ ]:
pos_data.to_excel('cftc先物ポジション_current.xlsx')

#### ここまで

In [ ]:
missed2 = []
i = 1
for date in ['20220517','20220524']:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/deacmesf"+date[-4:]+yyyy[-2:]+".htm"
    #try:
    texts = requests.get(url).text
    pattern = r'<!--ih:includeHTML file="deacmesf.txt"-->([^"]+)<!--/ih:includeHTML-->'
    texts = re.findall(pattern, texts)[0]
    texts = texts.split('\r\n')
    texts = [s for s in texts if (s != '')&(s != ' ')]
    if date == '20220517':
        texts[0] ='USD Malaysian Crude Palm Oil C - CHICAGO MERCANTILE EXCHANGE         Code-037021'
    pos = [item for i, element in enumerate(texts) if ' - CHICAGO' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
    nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
    for s in nested_list:
        try:
            temp = [int(s) for s in s[9].replace(',','').split(' ') if not s == ''][:2]
        except:
            print(s)
            break
            #print(temp)
        temp = pd.DataFrame([date,s[0]]+temp).T
        temp.columns=['date','commodity','long','short']
        pos_data = pd.concat([pos_data,temp],axis=0)
    #except:
    #    print(date)
    #    missed2 += [date]
    #    pass
    print(str(i)+'/'+str(len(tuesdays)))
    #clear_output(True)
    i += 1
print('終わりました')

In [ ]:
missed2

In [ ]:
pos_data.to_excel('cftc通貨先物ポジション_円.xlsx')

## テスト用

In [ ]:
tuesdays = ['20240326']

In [ ]:
#pos_data = pd.DataFrame(columns=['date','trader','commodity','long','short'])

missed = []
code_dic = {"a":""}
i = 1
for date in tuesdays:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        print('future')
        break
    #date = '20240319'
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/financial_lf"+date[-4:]+yyyy[-2:]+".htm"
    
    #url = 'https://www.cftc.gov/dea/futures/financial_lf.htm'
    texts = requests.get(url).text
    #print(texts)
    pattern = r'<!--ih:includeHTML file="financiallf.txt"-->([^"]+)<!--/ih:includeHTML-->'
    texts = re.findall(pattern, texts)[0].replace('$',' ')
    texts = texts.split('\r\n')
    texts = [s for s in texts if (s != '')&(s != ' ')]
        #if date == '20220517':
        #    texts[0] ='USD Malaysian Crude Palm Oil C - CHICAGO MERCANTILE EXCHANGE         Code-037021'

    pos = [item for i, element in enumerate(texts) if 'Traders in Financial Futures' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
    nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
    
    for s in nested_list:
        datas = [int(ss) for ss in s[9].replace(',','').split(' ') if not ss == '']
        #print(datas)
        traders = ['Dealer','AssetManager','LeveragedFunds','Other','NonRep']

        if (not s[6].startswith(' '))&(s[7].split('  ')[0] not in list(code_dic.keys())):
            #print(s[7].split('  ')[0])
            #print(s[6])
            code_dic[s[7].split('  ')[0]]=s[6]
        elif (s[6].startswith(' '))&(s[7].split('  ')[0] in list(code_dic.keys())):
            s[6] = code_dic[s[7]]
            #print(s[6])
            #break
        
        k = 0
        for j in range(0,len(datas),3):
            data = datas[j:j+2]
            #print(date,traders[k],s[6],data)
            temp = pd.DataFrame([date,traders[k],s[6]]+data).T
            temp.columns=['date','trader','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
            k += 1
    break
    print(str(i)+'/'+str(len(tuesdays)))
    clear_output(True)
    i += 1
print('終わりました')

In [ ]:
pos_data.shape

In [ ]:
pos_data.tail()